In [2]:
import tensorflow as tf

In [89]:
speaker_id = tf.placeholder(tf.int32, [None], 'speaker_id')

In [90]:
speaker_id

<tf.Tensor 'speaker_id_1:0' shape=(?,) dtype=int32>

In [6]:
num_inputs = 2
enc_prenet_sizes = [256, 128]
embed_size = enc_prenet_sizes[-1]
name = "before_highway"

In [7]:
embed_table = tf.get_variable(name, [num_inputs, embed_size], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.1))

In [9]:
before_highway = tf.nn.embedding_lookup(embed_table, speaker_id)

In [8]:
embed_table

<tf.Variable 'before_highway:0' shape=(2, 128) dtype=float32_ref>

In [10]:
before_highway

<tf.Tensor 'embedding_lookup/Identity:0' shape=(?, 128) dtype=float32>

In [12]:
PAD = '_'
EOS = '~'
PUNC = '!\'(),-.:;?'
SPACE = ' '

JAMO_LEADS = "".join([chr(_) for _ in range(0x1100, 0x1113)])
JAMO_VOWELS = "".join([chr(_) for _ in range(0x1161, 0x1176)])
JAMO_TAILS = "".join([chr(_) for _ in range(0x11A8, 0x11C3)])

VALID_CHARS = JAMO_LEADS + JAMO_VOWELS + JAMO_TAILS + PUNC + SPACE
ALL_SYMBOLS = PAD + EOS + VALID_CHARS

In [13]:
symbols = ALL_SYMBOLS # for korean

In [14]:
embedding_size = 256

In [16]:
char_embed_table = tf.get_variable('embedding', [len(symbols), embedding_size], dtype=tf.float32,initializer=tf.truncated_normal_initializer(stddev=0.5))

In [18]:
import numpy as np

np.shape(char_embed_table)

TensorShape([Dimension(80), Dimension(256)])

In [32]:
inputs = tf.placeholder(tf.int32, [None, None], 'inputs')

In [25]:
inputs.set_shape(inputs.shape)

In [34]:
char_embedded_inputs = tf.nn.embedding_lookup(char_embed_table, inputs)

In [35]:
char_embedded_inputs

<tf.Tensor 'embedding_lookup_2/Identity:0' shape=(?, ?, 256) dtype=float32>

In [31]:
#prenet
x = char_embedded_inputs
layer_sizes = [256, 128]

drop_rate = 0.5

for i, size in enumerate(layer_sizes):  # [f(256), f(128)]
    print (i)
    dense = tf.layers.dense(x, units=size, activation=tf.nn.relu, name='dense_%d' % (i+4))
    x = tf.layers.dropout(dense, rate=drop_rate,training=True, name='dropout_%d' % (i+4))
    print (x)

0
Tensor("dropout_4/dropout/mul_1:0", shape=(?, ?, 256), dtype=float32)
1
Tensor("dropout_5/dropout/mul_1:0", shape=(?, ?, 128), dtype=float32)


In [82]:
inputs = x

In [45]:
from tensorflow.python.layers import core

def conv1d(inputs, kernel_size, channels, activation, is_training, scope):
    with tf.variable_scope(scope):
        # strides=1, padding = same 이므로, kernel_size에 상관없이 크기가 유지된다.
        conv1d_output = tf.layers.conv1d(inputs,filters=channels,kernel_size=kernel_size,activation=activation,padding='same') # padding이 same이라 kenel size가 달라도 concat된다.
        return tf.layers.batch_normalization(conv1d_output, training=is_training)

bank_channel_size = 128
is_training= True

conv_fn = lambda k: conv1d(char_embedded_inputs, k, bank_channel_size, tf.nn.relu, is_training, 'conv1d_%d' % k)  # bank_channel_size =128

In [46]:
conv_fn

<function __main__.<lambda>>

In [47]:
bank_size = 16

conv_outputs = tf.concat( [conv_fn(k) for k in range(1, bank_size+1)], axis=-1,) 

Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


In [48]:
conv_outputs

<tf.Tensor 'concat:0' shape=(?, ?, 2048) dtype=float32>

In [51]:
# Maxpooling:
maxpool_width = 2
maxpool_output = tf.layers.max_pooling1d(conv_outputs,pool_size=maxpool_width,strides=1,padding='same')  # maxpool_width = 2

Instructions for updating:
Use keras.layers.MaxPooling1D instead.


In [69]:
maxpool_output

<tf.Tensor 'max_pooling1d/Squeeze:0' shape=(?, ?, 2048) dtype=float32>

In [70]:
proj_sizes= [128,128]
proj_width =3 
is_training = True

proj_out = maxpool_output
for idx, proj_size in enumerate(proj_sizes):   # [f(128), f(128)],  post: [f(256), f(80)]
    activation_fn = None if idx == len(proj_sizes) - 1 else tf.nn.relu
    proj_out = conv1d(proj_out, proj_width, proj_size, activation_fn,is_training, 'proj_{}'.format(idx + 3))  # proj_width = 3
    print (proj_out)

Tensor("proj_3/batch_normalization/batchnorm/add_1:0", shape=(?, ?, 128), dtype=float32)
Tensor("proj_4/batch_normalization/batchnorm/add_1:0", shape=(?, ?, 128), dtype=float32)


In [67]:
proj_out

<tf.Tensor 'max_pooling1d/Squeeze:0' shape=(?, ?, 2048) dtype=float32>

In [73]:
before_highway

<tf.Tensor 'embedding_lookup/Identity:0' shape=(?, 128) dtype=float32>

In [72]:
expanded_before_highway = tf.expand_dims(before_highway, [1])

In [75]:
expanded_before_highway

<tf.Tensor 'ExpandDims:0' shape=(?, 1, 128) dtype=float32>

In [76]:
tiled_before_highway = tf.tile(expanded_before_highway, [1, tf.shape(proj_out)[1], 1])

In [77]:
tiled_before_highway

<tf.Tensor 'Tile:0' shape=(?, ?, 128) dtype=float32>

In [79]:
proj_out

<tf.Tensor 'proj_4/batch_normalization/batchnorm/add_1:0' shape=(?, ?, 128) dtype=float32>

In [80]:
inputs

<tf.Tensor 'inputs_1:0' shape=(?, ?) dtype=int32>

In [83]:
highway_input = proj_out + inputs + tiled_before_highway

In [84]:
highway_input

<tf.Tensor 'add_2:0' shape=(?, ?, 128) dtype=float32>

In [86]:
def highwaynet(inputs, scope):
    highway_dim = int(inputs.get_shape()[-1])

    with tf.variable_scope(scope):
        H = tf.layers.dense(inputs,units=highway_dim, activation=tf.nn.relu,name='H')
        T = tf.layers.dense(inputs,units=highway_dim, activation=tf.nn.sigmoid,name='T',bias_initializer=tf.constant_initializer(-1.0))
        return H * T + inputs * (1.0 - T)
    
# 4-layer HighwayNet:
highway_depth = 4

for idx in range(highway_depth):
    highway_input = highwaynet(highway_input, 'highway_%d' % (idx+1))

In [87]:
highway_input

<tf.Tensor 'highway_4/add:0' shape=(?, ?, 128) dtype=float32>

In [99]:
rnn_input = highway_input

In [88]:
embed_table = tf.get_variable("encoder_rnn_init_state", [2, 128 * 2], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.1))

In [93]:
encoder_rnn_init_state = tf.nn.embedding_lookup(embed_table, speaker_id)

In [94]:
initial_state_fw, initial_state_bw = tf.split(encoder_rnn_init_state, 2, 1)

In [106]:
from tensorflow.contrib.rnn import GRUCell

rnn_size = 128
input_lengths = [-1,30]

cell_fw, cell_bw = GRUCell(rnn_size), GRUCell(rnn_size)

outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw,rnn_input,sequence_length=input_lengths, initial_state_fw=initial_state_fw,initial_state_bw=initial_state_bw,dtype=tf.float32)


Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [107]:
outputs

(<tf.Tensor 'bidirectional_rnn_4/fw/fw/transpose_1:0' shape=(?, ?, 128) dtype=float32>,
 <tf.Tensor 'ReverseSequence:0' shape=(2, ?, 128) dtype=float32>)

In [108]:
tf.concat(outputs, axis=2) 

<tf.Tensor 'concat_1:0' shape=(2, ?, 256) dtype=float32>